In [1]:
# Import PyTorch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision
import torchvision
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Import shutil for examining files
import shutil

# Import OS to access file path
import os

# Import Pandas and numpy for data
import numpy as np
import pandas as pd

# Import files from your computer
import tarfile
from PIL import Image

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


PyTorch version: 2.3.1+cu121
torchvision version: 0.18.1+cpu
CUDA available: True


In [5]:
# tar_file_path = r"C:\Users\Solomon's PC\supervised\Agriculture-Vision-2021.tar.gz"
# extraction_dir = r"C:\Users\Solomon's PC\supervised\extracted_data"

# os.makedirs(extraction_dir, exist_ok=True)

# with tarfile.open(tar_file_path, "r:gz") as tar:
#     tar.extractall(path=extraction_dir)

# print(f'Extracted data to {extraction_dir}')

Extracted data to C:\Users\Solomon's PC\supervised\extracted_data


In [2]:
root_dir = r"C:\Users\Solomon's PC\supervised\extracted_data\Agriculture-Vision-2021"


In [3]:
class AgricultureVisionDataset(Dataset):
    def __init__(self, root_dir, image_type='rgb', transform=None):
        self.root_dir = root_dir
        self.image_type = image_type
        self.transform = transform
        self.image_paths = sorted(os.listdir(os.path.join(root_dir, 'images', image_type)))
        self.mask_paths = sorted(os.listdir(os.path.join(root_dir, 'masks')))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'images', self.image_type, self.image_paths[idx])
        mask_name = os.path.join(self.root_dir, 'masks', self.mask_paths[idx])
        
        image = Image.open(img_name).convert("RGB")
        mask = Image.open(mask_name).convert("L")  # Assume masks are single-channel images
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

In [4]:
#Make a transformer
data_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

# Create dataset
train_dataset = AgricultureVisionDataset(root_dir=r"C:\Users\Solomon's PC\supervised\extracted_data\Agriculture-Vision-2021\train", transform=data_transforms)
val_dataset = AgricultureVisionDataset(root_dir=r"C:\Users\Solomon's PC\supervised\extracted_data\Agriculture-Vision-2021\val", transform=data_transforms)
test_dataset = AgricultureVisionDataset(root_dir=r"C:\Users\Solomon's PC\supervised\extracted_data\Agriculture-Vision-2021\test", transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [5]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.upconv1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        
        self.dec1 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.dec2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        
        upconv1 = self.upconv1(enc3)
        dec1 = self.dec1(torch.cat([enc2, upconv1], 1))
        
        upconv2 = self.upconv2(dec1)
        dec2 = self.dec2(torch.cat([enc1, upconv2], 1))
        
        return torch.sigmoid(self.final(dec2))

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 25

In [6]:
print(torch.cuda.is_available())

True


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # Validate the model
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

Epoch 1/25, Loss: 0.010782219398870564
Validation Loss: 0.0008566691206794365


In [ ]:
# Evaluation
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        test_loss += loss.item()
    
print(f"Test Loss: {test_loss/len(test_loader)}")

In [ ]:
torch.save(model.state_dict(), 'segmentation_model.pth')